In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv1D,BatchNormalization,Dropout

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

In [ ]:
credit =pd.read_csv("/content/Sample_Train3.csv")
print(credit.head())

   distance_from_home  distance_from_last_transaction  \
0           57.877857                        0.311140   
1           10.829943                        0.175592   
2            5.091079                        0.805153   
3            2.247564                        5.600044   
4           44.190936                        0.566486   

   ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                        1.945940                1          1   
1                        1.294219                1          0   
2                        0.427715                1          0   
3                        0.362663                1          1   
4                        2.222767                1          1   

   used_pin_number  online_order  fraud  
0                0             0      0  
1                0             0      0  
2                0             1      0  
3                0             1      0  
4                0             1      0  


In [ ]:
print(credit.isnull().sum())
print(credit['fraud'].value_counts())

distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64
fraud
0    18285
1     1715
Name: count, dtype: int64


In [ ]:
non_fraud = credit[credit['fraud'] == 0]
fraud = credit[credit['fraud'] == 1]
print(credit['fraud'].value_counts())

fraud
0    18285
1     1715
Name: count, dtype: int64


In [ ]:
print(non_fraud.shape, fraud.shape)
credit = pd.concat([fraud, non_fraud], ignore_index=True)
print(credit.head())

(18285, 8) (1715, 8)
   distance_from_home  distance_from_last_transaction  \
0            2.131956                       56.372401   
1            3.803057                       67.241081   
2           15.694986                      175.989182   
3           26.711462                        1.552008   
4           10.664474                        1.565769   

   ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                        6.358667                1          0   
1                        1.872950                1          0   
2                        0.855623                1          0   
3                        4.603601                1          1   
4                        4.886521                1          0   

   used_pin_number  online_order  fraud  
0                0             1      1  
1                0             1      1  
2                0             1      1  
3                0             1      1  
4                0             1   

In [ ]:
# now dividing the dataframe into dependent and independent variables
scaler = StandardScaler()
X = scaler.fit_transform(credit.drop(['fraud'], axis=1))
y = credit['fraud']

# check the shape
print(X.shape, y.shape)

(20000, 7) (20000,)


In [ ]:
class SalpSwarmAlgorithm:
    def __init__(self, objective_function, dim, population_size, max_iter):
        self.objective_function = objective_function
        self.dim = dim
        self.population_size = population_size
        self.max_iter = max_iter

    def run_with_logging(self):
        # Initialize population with random values between 0 and 1
        population = np.random.rand(self.population_size, self.dim)
        best_pos = np.zeros(self.dim)
        best_cost = float('inf')

        # Dictionary to store costs for each iteration
        costs = []

        for iter in range(self.max_iter):
            for i in range(self.population_size):
                # Calculate the cost for the current position
                cost = self.objective_function(population[i, :])

                # Update the best position and cost
                if cost < best_cost:
                    best_cost = cost
                    best_pos = population[i, :]

                # Log the cost for the current position
                costs.append((iter, cost, population[i, :]))

            # Perform SSA update (example, not actual implementation)
            # Update population positions based on SSA logic

            # Logging the best cost at each iteration
            print(f"Iter {iter + 1}/{self.max_iter}, Best Cost: {best_cost}")

        return best_pos, best_cost, costs


In [ ]:
def objective_function(feature_subset):
    selected_features = np.where(feature_subset > 0.5)[0]  # Use a threshold to select features
    if len(selected_features) == 0:
        return 1e6  # Penalize empty selection
    if len(selected_features) < 2:  # Example penalty for selecting too few features
        return 1e5

    # Convert X to a Pandas DataFrame for using .iloc
    X_df = pd.DataFrame(X)  # Create a DataFrame from the NumPy array X
    X_selected = X_df.iloc[:, selected_features]  # Now you can use .iloc
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=99)

    model = Sequential([
        Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], 1)),
        Dropout(0.5),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=5, verbose=0)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return 1 - accuracy


In [ ]:
# Parameters for SalpSwarmAlgorithm
dim = X.shape[1]  # Number of features
population_size = 30
max_iter = 10

# Instantiate and run the algorithm
ssa = SalpSwarmAlgorithm(objective_function, dim, population_size, max_iter)
best_pos, best_cost, costs = ssa.run_with_logging()

print(f"Best Position: {best_pos}")
print(f"Best Cost: {best_cost}")

Iter 1/10, Best Cost: 0.006749987602233887
Iter 2/10, Best Cost: 0.0065000057220458984
Iter 3/10, Best Cost: 0.0065000057220458984
Iter 4/10, Best Cost: 0.0065000057220458984
Iter 5/10, Best Cost: 0.0065000057220458984
Iter 6/10, Best Cost: 0.0065000057220458984
Iter 7/10, Best Cost: 0.0065000057220458984
Iter 8/10, Best Cost: 0.0065000057220458984
Iter 9/10, Best Cost: 0.0065000057220458984
Iter 10/10, Best Cost: 0.0065000057220458984
Best Position: [0.60150125 0.75453052 0.75402769 0.24250971 0.96736753 0.76544144
 0.9314709 ]
Best Cost: 0.0065000057220458984


In [ ]:
# Update the dataset with selected features
X_df = pd.DataFrame(X)  # Convert X to a Pandas DataFrame

# Calculate selected features based on best_pos (assuming best_pos is available from previous code)
threshold = 0.5
best_features_binary = best_pos > threshold
selected_features = np.where(best_features_binary)[0]

X_selected = X_df.iloc[:, selected_features] # Now you can use .iloc for indexing

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=99)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data to fit the CNN input requirements
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (16000, 6, 1)
Testing data shape: (4000, 6, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, AUC

# Define the model
model = Sequential()

# Input layer
model.add(Flatten(input_shape=(X_train.shape[1], 1)))

# Add hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model with additional metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), AUC()])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model on the test set
results = model.evaluate(X_test, y_test)

# Extract the metrics
test_loss = results[0]
test_accuracy = results[1]
test_precision = results[2]
test_recall = results[3]
test_auc = results[4]

# Calculate F1 score
test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test AUC: {test_auc}")
print(f"Test F1 Score: {test_f1_score}")


Model: "sequential_280"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_280 (Flatten)       (None, 6)                 0         
                                                                 
 dense_560 (Dense)           (None, 64)                448       
                                                                 
 dense_561 (Dense)           (None, 32)                2080      
                                                                 
 dropout_560 (Dropout)       (None, 32)                0         
                                                                 
 dense_562 (Dense)           (None, 64)                2112      
                                                                 
 dropout_561 (Dropout)       (None, 64)                0         
                                                                 
 dense_563 (Dense)           (None, 1)              